# Detectar arritmias cardíacas mediante señales de ECG parcialmente etiquetadas
### INF395 Introducción a las Redes Neuronales and Deep Learning
- Estudiante: Alessandro Bruno Cintolesi Rodríguez
- ROL: 202173541-0

## **1. Librerias**

In [ ]:
# === General / Utilidad ===
from datetime import datetime
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Optional, Dict

# === PyTorch / PyTorch Lightning ===
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.optim import AdamW
from torch.optim.lr_scheduler import LambdaLR, CosineAnnealingLR
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

# === Scikit-learn ===
from sklearn.cluster import KMeans
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
	f1_score, accuracy_score, recall_score, confusion_matrix,
	roc_auc_score, classification_report
)

## **2. Variables globales**

In [ ]:
# ============================================================
# Definición de las clases del dataset (ECG)
# ============================================================
CLASSES = {
	0: "(N) Latido normal",
	1: "(S) Latido supraventricular",
	2: "(V) Latido ventricular ectópico",
	3: "(F) Latido de fusión",
	4: "(Q) Latido desconocido"
}

# ============================================================
# Hiperparámetros de entrenamiento y configuración global
# ============================================================
SCHEDULER = "LambdaLR" # tipo de programador de tasa de aprendizaje ("LambdaLR" o "CosineAnnealingLR")

LOSS_LP = "FocalLoss"  # función de pérdida para LinearProbe (CrossEntropy o FocalLoss)
GAMMA_LP = 2.0         # parámetro gamma de Focal Loss (controla el foco en errores difíciles)

LOSS_FN = "FocalLoss"  # función de pérdida para FineTuning (CrossEntropy o FocalLoss)
GAMMA_FN = 2.0         # parámetro gamma de Focal Loss (controla el foco en errores difíciles)

SEED = 42              # semilla fija para reproducibilidad
SIGNALS = 187          # largo de cada señal ECG (número de muestras por ventana)
EMBEDDING_DIM = 256    # tamaño del embedding de salida del encoder (ResNet1D)
PROJ_HID = 256         # dimensión oculta del proyector (para TimeCLR)
PROJ_OUT = 128         # dimensión de salida del proyector (espacio contrastivo)

NUM_WORKERS = 0        # número de procesos paralelos para cargar datos (0 = main thread)
BATCH_SSL = 512        # tamaño de batch para entrenamiento auto-supervisado (TimeCLR)
BATCH_LP = 256         # tamaño de batch para LinearProbe
BATCH_FT = 256         # tamaño de batch para FineTuning

EPOCHS_SSL = 100       # número de épocas para auto-supervisado
EPOCHS_LP = 25         # número de épocas para LinearProbe
EPOCHS_FT = 25         # número de épocas para FineTuning

LR_SSL = 1e-3          # learning rate para etapa auto-supervisada
LR_LP = 1e-3           # learning rate para LinearProbe
LR_FT_HEAD = 5e-4      # learning rate para la cabeza del FineTuning
LR_FT_ENC = 3e-5       # learning rate para el encoder del FineTuning

WD = 5e-3              # weight decay (regularización L2)
TEMP = 0.15            # temperatura usada en la pérdida contrastiva NT-Xent

ArrayLike = np.ndarray # alias de tipo (para anotaciones de funciones y datasets)

## **3. Setup del Dispositivo**

In [ ]:
# Seteamos la semilla
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.seed_everything(SEED, workers=True)

# Seteamos el dispositivo
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True
print("Using device:", DEVICE)
if DEVICE.type == "cuda":
	print("GPU:", torch.cuda.get_device_name(0))

## **4. Funciones Auxiliares**

In [ ]:
def plot_ecg(X, y, aug=False, aug_title=""):
	y_class = CLASSES.get(y, "Sin clasificar")
	title = f"ECG Clase: {y_class}"
	if aug:
		title = title + f" | {aug_title}"

	plt.plot(X)
	plt.title(title)
	plt.xlabel("Tiempo (muestras)")
	plt.ylabel("Amplitud")
	plt.show()

In [ ]:
def plot_clusters(X, clusters, n_clusters):
	for i in range(n_clusters):
		cluster_mask = clusters == i
		cluster_mean = X[cluster_mask].mean(axis=0)
		plt.plot(cluster_mean, label=f"Cluster {i}")

	plt.legend()
	plt.title("Promedio de señal por cluster")
	plt.xlabel("Muestra")
	plt.ylabel("Amplitud promedio")
	plt.grid(True)
	plt.show()

## **5. Analisis Exploratorio de Datos**

In [ ]:
# Leemos nuestros datos desde los CSV
train_df = pd.read_csv("ecg_signals/train_semi_supervised.csv")
test_df = pd.read_csv("ecg_signals/test_semi_supervised.csv")

In [ ]:
# Dividimos nuestros datos en X (serie de tiempo) / y (label)
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

X_test = test_df.iloc[:, 1:-1].values
y_test = test_df.iloc[:, -1].values

In [ ]:
# Separamos nuestros datos de entrenamiento por etiquetados (labeled) y no etiquetados (unlabeled)
y_train_np = np.asarray(y_train)
mask_unl = np.isnan(y_train_np.astype(float, copy=False)) if y_train_np.dtype.kind in "fc" else np.zeros_like(y_train_np, dtype=bool)
mask_unl |= (y_train_np == -1)

mask_lab = ~mask_unl

X_lab_train = X_train[mask_lab]
y_lab_train = y_train_np[mask_lab].astype(int)
X_unl_train = X_train[mask_unl]

num_classes = int(np.unique(y_lab_train).size)
T = X_train.shape[1] 

In [ ]:
# Graficando un ECG
plot_ecg(X=X_train[1], y=y_train[1])

In [ ]:
# Clusters con K-means
train_kmeans = KMeans(n_clusters=5, random_state=SEED)
train_clusters = train_kmeans.fit_predict(X_train)

In [ ]:
plot_clusters(X=X_train, clusters=train_clusters, n_clusters=5)

In [ ]:
# ============================================================
# Conteo de muestras por clase en el conjunto de entrenamiento
# ============================================================
unique, counts = np.unique(y_train, return_counts=True)
for c, n in zip(unique, counts):
	if not np.isnan(c):
		c = int(c)
	print(f"Clase {c}: {n} muestras ({n/len(y_train)*100:.2f}%)")

In [ ]:
n_classes = 5 # número total de clases del problema

# ============================================================
# 1) Preparación de las etiquetas válidas
# ============================================================
y_valid = y_train[~np.isnan(y_train)]       # elimina valores NaN (no etiquetados)
y_valid = y_valid[y_valid >= 0].astype(int) # elimina etiquetas negativas y las convierte a enteros
print("Clases en y_valid:", np.unique(y_valid))

class_counts = np.bincount(y_valid, minlength=n_classes)
print("Cantidad por clase (0..4):", class_counts)

# Pesos base: 1 / sqrt(freq)
inv_sqrt = 1.0 / np.sqrt(class_counts + 1e-8)
inv_sqrt = inv_sqrt / inv_sqrt.mean()
print("Pesos 1/sqrt(freq) normalizados:", inv_sqrt)

# Suavizado extra
alpha = 0.5
class_weights_np = inv_sqrt ** alpha
class_weights_np = class_weights_np / class_weights_np.mean()
print("Pesos por clase suavizados para LOSS (0..4):", class_weights_np)

# Tensor PARA LA LOSS (float32)
class_weights_loss = torch.tensor(class_weights_np, dtype=torch.float32)

## **6. Modelo basado en TimeCLR**

### **6.1 Augmentaciones del Modelo**

La clase ECGTimeAugment define un conjunto de transformaciones aleatorias para señales ECG (series temporales), usadas como data augmentation durante el entrenamiento. Cada método aplica una alteración leve o moderada para generar variaciones realistas de la señal original:
- Jitter: añade ruido gaussiano (simula ruido del sensor).
- Scaling: cambia levemente la amplitud (simula diferencias en ganancia).
- Time Mask: enmascara un fragmento corto de la señal (similar a cutout o SpecAugment).
- Crop: recorta y reescala un segmento temporal (simula distintas duraciones o desplazamientos).
- El método __call__ ejecuta estas transformaciones con probabilidades independientes (p_jitter, p_scaling, etc.), devolviendo una versión aumentada de la señal.

En resumen: esta clase genera variaciones sintéticas y realistas de señales ECG, mejorando la robustez y generalización del modelo.

In [ ]:
# Clase que aplica aumentaciones temporales a señales ECG (para robustecer el modelo)
class ECGTimeAugment:
	def __init__(
		self,
		series_len: int = SIGNALS,                               # longitud esperada de la señal
		p_jitter: float = 0.7,		jitter_sigma: float = 0.008, # probabilidad y nivel de ruido
		p_scaling: float = 0.6,		scaling_sigma: float = 0.05, # probabilidad y nivel de escalado
		p_tmask: float = 0.3,		tmask_frac=(0.02, 0.06),     # probabilidad y fracción del tiempo a enmascarar	
		p_crop: float = 0.5,		crop_frac=(0.8, 0.98),       # probabilidad y tamaño relativo del recorte
		use_perm: bool = False                                   # bandera para futuros usos (permute segments)
	):
		# Guarda los parámetros de configuración
		self.T = series_len
		self.p_jitter = p_jitter; self.jitter_sigma = jitter_sigma
		self.p_scaling = p_scaling; self.scaling_sigma = scaling_sigma
		self.p_tmask = p_tmask; self.tmask_frac = tmask_frac
		self.p_crop = p_crop; self.crop_frac = crop_frac
		self.use_perm = use_perm

	# --- 1. Jitter: agrega ruido gaussiano a la señal ---
	def _jitter(self, x):
		y = x + np.random.normal(0.0, self.jitter_sigma, size=x.shape)
		return y.astype(x.dtype, copy=False)

	# --- 2. Scaling: multiplica la señal por un factor aleatorio ---
	def _scaling(self, x):
		y = x * np.random.normal(1.0, self.scaling_sigma)
		return y.astype(x.dtype, copy=False)

	# --- 3. Time Mask: pone a cero una ventana temporal aleatoria ---
	def _time_mask(self, x):
		y = x.astype(np.float32, copy=False).copy()
		# ancho de la máscara (en número de puntos)
		w = max(1, int(self.T * np.random.uniform(*self.tmask_frac)))
		# posición inicial del segmento a enmascarar
		s = np.random.randint(0, max(1, self.T - w + 1))
		y[s:s+w] = 0.0 # "borra" esa parte de la señal
		return y.astype(x.dtype, copy=False)

	# --- 4. Crop: recorta un segmento y lo reescala al largo original ---
	def _crop(self, x):
		# define el ancho del recorte (entre 80% y 98% del total por defecto)
		w = max(2, int(self.T * np.random.uniform(*self.crop_frac)))
		# elige una posición inicial aleatoria
		s = np.random.randint(0, max(1, self.T - w + 1))
		seg = x[s:s+w] # segmento recortado
		# reinterpolamos el segmento al largo original (T)
		i_old = np.linspace(0, 1, num=w)
		i_new = np.linspace(0, 1, num=self.T)
		y = np.interp(i_new, i_old, seg) # interpolación lineal
		return y.astype(x.dtype, copy=False)

	# --- 5. Lógica principal: aplica las transformaciones con probabilidad ---
	def __call__(self, x):
		y = x
		# Cada augment se aplica con su respectiva probabilidad
		if np.random.rand() < self.p_crop:
			y = self._crop(y)
		if np.random.rand() < self.p_tmask:
			y = self._time_mask(y)
		if np.random.rand() < self.p_scaling:
			y = self._scaling(y)
		if np.random.rand() < self.p_jitter:
			y = self._jitter(y)

		# Devuelve la señal transformada como un array contiguo en memoria
		return np.ascontiguousarray(y, dtype=np.float32)

# Instancia del aumentador, indicando la longitud de las señales ECG
ecgtime_augment = ECGTimeAugment(series_len=SIGNALS)

In [ ]:
aug = ecgtime_augment(X_train[1])
plot_ecg(X=aug, y=y_train[1], aug=True, aug_title="Augmentación")

### **6.2 Dataset del Modelo**

Estas tres clases definen datasets especializados para señales ECG (series temporales), adaptados a distintos escenarios de entrenamiento:
1. TimeCLRDataset → se usa para aprendizaje contrastivo no supervisado (como TimeCLR):
	- Cada muestra genera dos vistas aumentadas (a1, a2) de la misma señal aplicando transformaciones aleatorias.
	- Ambas vistas se normalizan (z-score) y se devuelven como tensores (1, T) para el cálculo de pérdidas contrastivas (como nt_xent_loss).
2. LabeledECGDataset → se usa para entrenamiento supervisado:
	- Cada señal tiene una etiqueta (y).
	- Con cierta probabilidad (p_aug), se aplica una transformación temporal (augmentación).
	- Luego se normaliza y convierte a tensor, devolviendo (x, y) para clasificación.
3. UnlabeledECGDataset → versión simplificada sin etiquetas, usada en evaluación o inferencia:
	- Solo normaliza y convierte las señales a tensores.

En resumen: Estas clases estructuran los datos ECG para distintos tipos de aprendizaje —contrastivo, supervisado o sin etiquetas— garantizando normalización consistente, formato correcto y compatibilidad directa con DataLoader de PyTorch.

In [ ]:
# ============================================================
# Dataset para entrenamiento contrastivo (TimeCLR)
# ============================================================
class TimeCLRDataset(Dataset):
	def __init__(self, X, transform=None, eps=1e-6):
		# Asegura que X sea un array float32
		X = np.asarray(X, dtype=np.float32)

		# Soporta dos formatos: (N, T) o (N, 1, T)
		if X.ndim == 2:
			self.X = X
		elif X.ndim == 3 and X.shape[1] == 1:
			self.X = X[:, 0, :] # elimina la dimensión de canal
		else:
			raise ValueError("X debe ser (N, T) o (N, 1, T)")
		
		# Guarda longitud de la serie temporal
		self.T = self.X.shape[1]
		self.transform = transform # función de augmentación temporal
		self.eps = eps # evita divisiones por cero en z-score

	def __len__(self):
		# Retorna el número de muestras del dataset
		return self.X.shape[0]

	def _z(self, x):
		# Normaliza por z-score: (x - media) / desviación estándar
		m, s = x.mean(), x.std()
		return (x - m) / (s + self.eps)

	def __getitem__(self, idx: int):
		# Obtiene la señal correspondiente
		x = self.X[idx]

		# Aplica dos transformaciones independientes (vistas augmentadas)
		a1 = self.transform(x) if self.transform else x
		a2 = self.transform(x) if self.transform else x

		# Normaliza ambas vistas
		a1 = self._z(a1).astype(np.float32, copy=False)
		a2 = self._z(a2).astype(np.float32, copy=False)

		# Asegura que sean contiguas en memoria (requerido por torch.from_numpy)
		a1 = np.ascontiguousarray(a1)
		a2 = np.ascontiguousarray(a2)

		# Convierte a tensores y agrega dimensión de canal (1, T)
		x1 = torch.from_numpy(a1).unsqueeze(0)
		x2 = torch.from_numpy(a2).unsqueeze(0)

		# Devuelve el par de vistas aumentadas
		return x1, x2

# ============================================================
# Dataset etiquetado (para entrenamiento supervisado)
# ============================================================
class LabeledECGDataset(Dataset):
	def __init__(self, X, y, eps=1e-6, transform=None, p_aug=0.0):
		# Guarda señales y etiquetas como arrays numpy
		self.X = np.asarray(X, dtype=np.float32)
		self.y = np.asarray(y, dtype=np.int64)

		self.eps = eps
		self.transform = transform # posible augmentación
		self.p_aug = p_aug # probabilidad de aplicarla

	def __len__(self):
		# Número de ejemplos en el dataset
		return self.X.shape[0]

	def _z(self, x):
		# Normalización z-score
		m, s = x.mean(), x.std()
		return (x - m) / (s + self.eps)

	def __getitem__(self, i):
		# Obtiene la señal i-ésima
		x = self.X[i]

		# Aplica augmentación con probabilidad p_aug
		if self.transform is not None and np.random.rand() < self.p_aug:
			x = self.transform(x)

		# Normaliza y convierte a tensor
		x = self._z(x)
		x = np.ascontiguousarray(x, dtype=np.float32)
		x = torch.from_numpy(x).unsqueeze(0)

		# Convierte la etiqueta a tensor
		y = torch.tensor(self.y[i])

		# Retorna el par (señal, etiqueta)
		return x, y

# ============================================================
# Dataset no etiquetado (para inferencia o pseudo-labeling)
# ============================================================
class UnlabeledECGDataset(Dataset):
	def __init__(self, X, eps=1e-6):
		# Guarda solo las señales
		self.X = np.asarray(X, dtype=np.float32)
		self.eps = eps

	def __len__(self):
		# Devuelve el número de señales
		return self.X.shape[0]

	def _z(self, x):
		# Normaliza por z-score
		m, s = x.mean(), x.std()
		return (x - m) / (s + self.eps)

	def __getitem__(self, i):
		# Normaliza y convierte la señal i-ésima a tensor
		x = self._z(self.X[i])
		x = np.ascontiguousarray(x, dtype=np.float32)
		x = torch.from_numpy(x).unsqueeze(0)
		return x

### **6.3 Backbone del Modelo**

Este ``ResNet1DBackbone`` es un extractor de características para series temporales basado en bloques residuales (``ResBlocks``) de convoluciones 1D:
- Cada ``ResBlock1D`` aplica dos convoluciones 1D con normalización (``BatchNorm``) y activación ``ReLU``, además de una conexión residual que permite sumar la entrada original al resultado (facilita el flujo del gradiente y evita el desvanecimiento).
- Si las dimensiones cambian (por stride o número de canales), usa un atajo (shortcut) con una convolución 1×1 para ajustar tamaño.
- El ``ResNet1DBackbone`` encadena varios de estos bloques, aumentando el número de canales y la dilatación progresivamente para capturar dependencias a distintas escalas temporales.
- Finalmente, usa un ``Adaptive Average Pooling`` para resumir toda la señal en un vector y una capa lineal (fc) para proyectarlo al espacio de embedding (emb_dim), entregando una representación compacta del input.

En resumen: es una red residual 1D que transforma una señal temporal en un vector de características robusto y multiescala.

In [ ]:
# Bloque residual 1D (usa convoluciones sobre series temporales)
class ResBlock1D(nn.Module):
	def __init__(self, in_ch, out_ch, k=7, stride=1, dilation=1):
		super().__init__()
		# Calcula padding para mantener el tamaño (depende de kernel y dilatación)
		p = (k//2) * dilation

		# Primera convolución: puede reducir la longitud temporal si stride > 1
		self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=k, stride=stride, padding=p, dilation=dilation, bias=False)
		self.bn1 = nn.BatchNorm1d(out_ch)

		# Segunda convolución: mantiene tamaño (stride=1)
		self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=k, stride=1, padding=p, dilation=dilation, bias=False)
		self.bn2 = nn.BatchNorm1d(out_ch)

		# Shortcut (conexión residual):
		# Si las dimensiones cambian, usa conv1x1 para ajustar canales o stride.
		# Si no cambian, deja la identidad.
		self.shortcut = nn.Sequential(
			nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=stride, bias=False),
			nn.BatchNorm1d(out_ch)
		) if (in_ch != out_ch or stride != 1) else nn.Identity()

		# Activación
		self.relu = nn.ReLU(inplace=True)

	def forward(self, x):
		# Camino principal
		y = self.relu(self.bn1(self.conv1(x)))
		y = self.bn2(self.conv2(y))

		# Suma residual (x + F(x))
		y = y + (x if isinstance(self.shortcut, nn.Identity) else self.shortcut(x))

		# Activación final
		return self.relu(y)

# Red residual completa para extracción de embeddings 1D
class ResNet1DBackbone(nn.Module):
	def __init__(self, in_ch=1, emb_dim=256, widths=(64,128,256,256), dilations=(1,2,4,8)):
		super().__init__()
		layers = []
		ch = in_ch # canales de entrada (ej. 1 para señales univariadas)

		# Construye múltiples etapas (bloques residuales) con distinta anchura y dilatación
		for w, d in zip(widths, dilations):
			# Primer bloque de la etapa: puede reducir la resolución temporal (stride=2)
			layers += [
				ResBlock1D(ch, w, k=7, stride=2, dilation=d),
				# Segundo bloque: mantiene tamaño (stride=1)
				ResBlock1D(w, w, k=7, stride=1, dilation=d)
			]
			ch = w # actualiza número de canales para la siguiente etapa

		# Secuencia completa de extracción de características
		self.feat = nn.Sequential(
			*layers,
			nn.AdaptiveAvgPool1d(1) # colapsa dimensión temporal a 1 (pooling global)
		)

		# Capa lineal final para obtener embedding de tamaño emb_dim
		self.fc = nn.Linear(ch, emb_dim)

	def forward(self, x):
		# Extrae características convolucionales
		h = self.feat(x).squeeze(-1) # salida de tamaño (batch, canales)

		# Proyección al espacio de embedding
		return self.fc(h)

### **6.4 ProjectionHead del Modelo**

La clase ``ProjectionHead`` es una cabeza de proyección usada comúnmente en modelos de aprendizaje contrastivo (como SimCLR o TimeCLR):
- Toma un vector de características (in_dim) producido por un backbone (por ejemplo, el ``ResNet1DBackbone``).
- Lo pasa por una pequeña MLP de dos capas lineales con normalización (``BatchNorm1d``) y activación ``ReLU``.
- La primera capa expande o transforma el espacio intermedio (hid_dim), y la segunda lo proyecta al espacio de embedding contrastivo (out_dim), donde se calculan las similitudes entre muestras.

En resumen: convierte los embeddings del backbone en representaciones normalizadas y compactas ideales para optimizar una pérdida contrastiva (como InfoNCE).

In [ ]:
# Cabeza de proyección usada en aprendizaje contrastivo (p. ej. SimCLR, TimeCLR)
class ProjectionHead(nn.Module):
	def __init__(self, in_dim: int, hid_dim: int = 256, out_dim: int = 128):
		super().__init__()

		# Definimos una red totalmente conectada (MLP) de dos capas
		self.net = nn.Sequential(
			# Capa lineal inicial: transforma el embedding del backbone a un espacio intermedio
			nn.Linear(in_dim, hid_dim, bias=False),

			# Normalización por lotes para estabilizar el entrenamiento
			nn.BatchNorm1d(hid_dim),

			# Activación no lineal ReLU
			nn.ReLU(inplace=True),

			# Segunda capa lineal: proyecta al espacio de embedding final (para contraste)
			nn.Linear(hid_dim, out_dim, bias=True)
		)

	# Propagación hacia adelante
	def forward(self, x: torch.Tensor) -> torch.Tensor:
		# Aplica la red definida (MLP) al embedding de entrada
		return self.net(x)

### **6.5 Clasificador del Modelo**

La clase ``ClassifierHead`` es una cabeza de clasificación totalmente conectada que transforma el embedding del backbone en predicciones de clase:
- Primero aplica una normalización por capa (``LayerNorm``) para estabilizar las entradas.
- Luego pasa por una capa lineal + ``ReLU``, que aprende combinaciones no lineales de las características.
- Se añade un ``Dropout`` (p_drop) para reducir el overfitting.
- Finalmente, una última capa lineal proyecta al número de clases (n_classes).

En resumen: convierte el embedding del modelo base en logits de clasificación, listos para una función de pérdida como ``CrossEntropy`` o ``Focal Loss``.

In [ ]:
# Cabeza de clasificación totalmente conectada (MLP simple)
class ClassifierHead(nn.Module):
	def __init__(self, in_dim=256, hid=256, n_classes=5, p_drop=0.4):
		super().__init__()

		# Definimos una pequeña red secuencial
		self.net = nn.Sequential(
			# Normalización por capa: estabiliza las activaciones de entrada
			nn.LayerNorm(in_dim),

			# Capa lineal: transforma el embedding de entrada al espacio oculto
			nn.Linear(in_dim, hid),

			# Activación ReLU: introduce no linealidad
			nn.ReLU(inplace=True),

			# Dropout: apaga aleatoriamente neuronas durante el entrenamiento (reduce overfitting)
			nn.Dropout(p_drop),

			# Capa lineal final: proyecta al número de clases (produce los logits)
			nn.Linear(hid, n_classes),
		)

	# Propagación hacia adelante: pasa el embedding por la red definida
	def forward(self, x):
		return self.net(x)

### **6.6 LossFunction del Modelo**

##### **6.6.a NTXentLoss**

La función ``nt_xent_loss`` implementa la ``NT-Xent Loss`` (Normalized Temperature-scaled Cross-Entropy Loss) usada en modelos contrastivos como ``SimCLR``:
- Entrada: dos lotes de embeddings (z1, z2) que representan diferentes vistas (augmentaciones) de las mismas muestras.
- Primero normaliza los vectores para que su similitud se mida solo por el ángulo (coseno).
- Luego concatena ambos lotes y calcula la matriz de similitudes coseno escalada por una temperatura (temperature) que controla la dispersión.
- Los pares positivos son las posiciones correspondientes entre z1 y z2, y todos los demás pares actúan como negativos.
- Finalmente aplica cross-entropy para maximizar la similitud de los positivos y minimizar la de los negativos.

En resumen: esta función enseña al modelo a acercar representaciones de vistas del mismo ejemplo y alejar las de distintos ejemplos, formando un espacio de embeddings discriminativo.

In [ ]:
# NT-Xent Loss (Normalized Temperature-scaled Cross Entropy Loss)
# Usada en aprendizaje contrastivo (por ejemplo, SimCLR)
def nt_xent_loss(z1: torch.Tensor, z2: torch.Tensor, temperature: float = 0.2) -> torch.Tensor:
	# --- 1. Normalización ---
	# Normaliza los embeddings por fila (cada vector con norma 1)
	# Esto hace que la similitud se base solo en el ángulo entre vectores (cosine similarity)
	z1 = F.normalize(z1, dim=1)
	z2 = F.normalize(z2, dim=1)

	# Tamaño del batch
	B = z1.size(0)

	# --- 2. Concatenación ---
	# Combina ambos lotes (2B muestras en total)
	Z = torch.cat([z1, z2], dim=0)

	# --- 3. Precisión ---
	# Si los tensores están en media precisión (fp16 o bf16),
	# se convierten a float32 para evitar errores numéricos en el producto matricial
	if Z.dtype in (torch.float16, torch.bfloat16):
		Zm = Z.float()
	else:
		Zm = Z

	# --- 4. Similitud ---
	# Calcula la matriz de similitudes coseno (producto punto) entre todos los pares
	# y la escala por la "temperatura", que ajusta la suavidad de las probabilidades
	sim = torch.matmul(Zm, Zm.t()) / temperature # (2B x 2B)

	# --- 5. Máscara de auto-similitud ---
	# Crea una máscara identidad para eliminar la comparación de cada muestra consigo misma
	mask = torch.eye(2 * B, dtype=torch.bool, device=Z.device)
	sim = sim.masked_fill(mask, float('-inf'))

	# --- 6. Targets ---
	# Define los índices de los pares positivos:
	# - la muestra i en z1 debe emparejarse con la muestra i en z2
	# - y viceversa
	target = torch.cat([
		torch.arange(B, 2 * B, device=Z.device), # pares z1 → z2
		torch.arange(0, B, device=Z.device) # pares z2 → z1
	], dim=0)

	# --- 7. Pérdida ---
	# Aplica cross-entropy sobre la matriz de similitudes:
	# maximiza la similitud con el par positivo y minimiza con los negativos
	loss = F.cross_entropy(sim, target)

	# Devuelve la pérdida promedio
	return loss

##### **6.6.b FocalLoss**

La clase ``FocalLoss`` implementa la funcion de pérdida ``Focal Loss``, una variante de la pérdida de entropía cruzada diseñada para manejar datasets con clases desbalanceadas:
- Idea principal: penaliza más los ejemplos difíciles y reduce el peso de los fáciles, controlado por el parámetro γ (gamma).
- Primero calcula la probabilidad logarítmica con log_softmax y la pérdida base cross-entropy sin reducción.
- Luego obtiene la probabilidad pt asociada a la clase verdadera.
- El término (1 - pt) ** gamma reduce la contribución de las muestras fáciles (cuando pt es alto).
- Finalmente promedia el resultado para devolver una pérdida escalar.

En resumen: ``FocalLoss`` enfoca el entrenamiento en los ejemplos difíciles y es muy útil para problemas donde una o más clases están subrepresentadas.

In [ ]:
# Implementación de la Focal Loss
# Se usa para problemas con clases desbalanceadas o donde hay muchas muestras fáciles.
class FocalLoss(torch.nn.Module):
	def __init__(self, gamma=2.0, weight=None):
		super().__init__()
		# gamma controla cuánto se reduce la pérdida de las muestras fáciles
		# valores mayores a 0 hacen que el modelo se enfoque más en los casos difíciles
		self.gamma = gamma

		# weight permite asignar un peso distinto a cada clase (útil si están desbalanceadas)
		self.weight = weight
	def forward(self, logits, target):
		# --- 1. Cálculo de probabilidades logarítmicas ---
		# Convierte los logits (salidas sin normalizar) en log-probabilidades con softmax
		logp = F.log_softmax(logits, dim=1)

		# --- 2. Obtiene las probabilidades estándar ---
		# p = e^(logp) → probabilidades normales
		p = logp.exp()

		# --- 3. Calcula la pérdida de entropía cruzada estándar ---
		# nll_loss devuelve la pérdida por muestra (sin promediar)
		ce = F.nll_loss(logp, target, reduction="none", weight=self.weight)

		# --- 4. Obtiene la probabilidad de la clase correcta ---
		# Extrae p_t: la probabilidad asignada a la clase verdadera para cada muestra
		pt = p.gather(1, target.unsqueeze(1)).squeeze(1)

		# --- 5. Aplica el factor focal ---
		# (1 - pt)^gamma reduce el peso de las muestras fáciles (cuando pt ≈ 1)
		# Multiplica por la pérdida de entropía cruzada
		loss = (1 - pt) ** self.gamma * ce

		# --- 6. Retorna la pérdida promedio del batch ---
		return loss.mean()

### **6.7 Arquitectura del Modelo**

La clase ``TimeCLRModel`` define un modelo auto-supervisado basado en ``PyTorch Lightning`` para aprender representaciones de señales temporales (como ECG) usando el enfoque contrastivo ``TimeCLR``.
- Encoder: usa un ``ResNet1DBackbone`` para extraer embeddings de las señales.
- Projector: aplica una MLP (``ProjectionHead``) que transforma los embeddings al espacio donde se calcula la pérdida contrastiva.
- training_step: recibe dos vistas aumentadas de la misma señal (x1, x2), obtiene sus proyecciones (z1, z2) y calcula la pérdida ``NT-Xent``, que acerca los pares positivos y aleja los negativos.
- configure_optimizers: configura el optimizador ``AdamW`` y un programador de tasa de aprendizaje (warmup + cosine decay o CosineAnnealingLR).
- encode: método auxiliar para obtener embeddings normalizados del encoder (sin pasar por la cabeza de proyección).

En resumen: ``TimeCLRModel`` entrena una red 1D tipo ResNet a aprender representaciones invariantes a las aumentaciones temporales, que luego pueden reutilizarse en tareas supervisadas como clasificación ECG.

In [ ]:
# ============================================================
# Modelo auto-supervisado basado en TimeCLR
# Aprende representaciones contrastivas de señales temporales (ej: ECG)
# ============================================================
class TimeCLRModel(pl.LightningModule):
	def __init__(
		self,
		emb_dim: int = EMBEDDING_DIM, # dimensión del embedding del encoder
		proj_hid: int = PROJ_HID,     # tamaño oculto del proyector
		proj_out: int = PROJ_OUT,     # dimensión del espacio contrastivo
		temperature: float = TEMP,    # parámetro de temperatura para NT-Xent
		lr: float = LR_SSL,           # tasa de aprendizaje
		weight_decay: float = WD      # regularización L2
	):
		super().__init__()
		self.save_hyperparameters() # guarda los hiperparámetros en el checkpoint

		# Encoder convolucional 1D (ej. ResNet1DBackbone)
		self.encoder = ResNet1DBackbone(in_ch=1, emb_dim=emb_dim)

		# Cabeza de proyección (MLP) que lleva los embeddings al espacio contrastivo
		self.projector = ProjectionHead(emb_dim, proj_hid, proj_out)
		
		# Parámetros del entrenamiento contrastivo
		self.temperature = temperature
		self.lr = lr
		self.weight_decay = weight_decay

	# ------------------------------------------------------------
	# Forward: pasa por el encoder y luego por la cabeza de proyección
	# ------------------------------------------------------------
	def forward(self, x: torch.Tensor) -> torch.Tensor:
		h = self.encoder(x) # embedding del backbone
		z = self.projector(h) # proyección al espacio contrastivo
		return z

	# ------------------------------------------------------------
	# Paso de entrenamiento (batch: contiene 2 vistas aumentadas)
	# ------------------------------------------------------------
	def training_step(self, batch, batch_idx):
		x1, x2 = batch # dos vistas del mismo ejemplo
		z1 = self(x1) # pasa la primera vista
		z2 = self(x2) # pasa la segunda vista

		# Calcula la pérdida contrastiva (NT-Xent)
		loss = nt_xent_loss(z1, z2, self.temperature)

		# Registra la pérdida en barra de progreso y logs
		self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
		return loss

	# ------------------------------------------------------------
	# Configura el optimizador y el scheduler del LR
	# ------------------------------------------------------------
	def configure_optimizers(self):
		# Optimizador AdamW (Adam con weight decay separado)
		opt = AdamW(self.parameters(), lr=self.lr, weight_decay=self.weight_decay)

		# Calcula cantidad total de pasos por época
		if self.trainer.datamodule:
			steps_per_epoch = len(self.trainer.datamodule.train_dataloader())
		else:
			steps_per_epoch = len(self.trainer.fit_loop._data_source.dataloader())

		
		total_steps = self.trainer.max_epochs * steps_per_epoch
		warmup_steps = int(0.1 * total_steps) # 10% de pasos de warmup

		# Función que define el comportamiento del LR
		def warmup_then_cosine(step):
			if step < warmup_steps: # fase de warmup
				return step / max(1, warmup_steps)
			progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
			# decaimiento coseno
			return 0.5 * (1 + math.cos(math.pi * progress))

		# Selecciona el scheduler según configuración global
		if SCHEDULER == "LambdaLR":
			scheduler = LambdaLR(opt, lr_lambda=warmup_then_cosine)
		elif SCHEDULER == "CosineAnnealingLR":
			scheduler = CosineAnnealingLR(opt, T_max=self.trainer.max_epochs)
		else:
			raise ValueError(f"Scheduler desconocido: {SCHEDULER}")

		# Devuelve optimizador y scheduler integrados en formato Lightning
		return {"optimizer": opt, "lr_scheduler": {"scheduler": scheduler, "interval": "step"}}

	# ------------------------------------------------------------
	# Método para codificar señales sin entrenar (modo evaluación)
	# ------------------------------------------------------------
	@torch.no_grad()
	def encode(self, x: torch.Tensor, normalize: bool = True) -> torch.Tensor:
		h = self.encoder(x) # genera embedding
		if normalize: # opcionalmente lo normaliza
			h = F.normalize(h, dim=1)
		return h

### **6.7 Pre-Entrenamiento del Modelo**

Este bloque de código entrena el modelo ``TimeCLR`` de forma auto-supervisada usando señales ECG:
1. ``TimeCLRDataset`` y ``DataLoader``: Se crea un dataset (ssl_ds) con las señales X_train, aplicando la función de aumentación ecgtime_augment para generar vistas distintas de cada ejemplo. El DataLoader (ssl_dl) sirve para alimentar los lotes al modelo durante el entrenamiento contrastivo.
2. ``TimeCLRModel``: Se instancia el modelo auto-supervisado con su encoder (ResNet1D) y cabeza de proyección, configurando hiperparámetros como la temperatura, LR y regularización.
3. Entrenamiento con Trainer: Se configura un Trainer de PyTorch Lightning con GPU (si está disponible), mixed precision (float16), clipping de gradiente y monitoreo del learning rate. Luego se entrena (trainer.fit) el modelo con los datos aumentados (ssl_dl).
4. encoder = timeclr_model.encoder.eval(): Al final, se extrae el encoder ya entrenado (sin la cabeza contrastiva) y se pone en modo evaluación (.eval()), listo para usarse en tareas supervisadas posteriores (por ejemplo, clasificación de ritmos cardíacos).

En resumen: este código entrena un encoder auto-supervisado que aprende representaciones robustas de ECG sin usar etiquetas.

In [ ]:
# Dataset auto-supervisado (TimeCLR) con aumentaciones
X_all_train = X_train
ssl_ds = TimeCLRDataset(
	X=X_all_train,            # matrices de señales (N, T) o (N, 1, T)
	transform=ecgtime_augment # función de augmentación temporal (jitter, crop, etc.)
)

# DataLoader para muestrear batches y barajarlos en cada época
ssl_dl = DataLoader(
	ssl_ds,
	batch_size=BATCH_SSL,   # tamaño de lote para contraste
	shuffle=True,           # mezcla ejemplos cada época
	num_workers=NUM_WORKERS # workers para cargar datos en paralelo
)

# Modelo TimeCLR: encoder (ResNet1D) + cabeza de proyección (MLP)
timeclr_model = TimeCLRModel(
	emb_dim=EMBEDDING_DIM, # dimensión del embedding del encoder
	proj_hid=PROJ_HID,     # ancho oculto del projector
	proj_out=PROJ_OUT,     # dim. del espacio contrastivo
	temperature=TEMP,      # temperatura NT-Xent
	lr=LR_SSL,             # learning rate para AdamW
	weight_decay=WD        # regularización L2
)

# Cálculo auxiliar de pasos (para schedulers con warmup, monitoreo, etc.)
steps_per_epoch = max(1, len(ssl_dl))
warmup_steps = int(0.1 * EPOCHS_SSL * steps_per_epoch) # 10% de warmup
lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval="step") # registra LR por paso

# Trainer de PyTorch Lightning: configura hardware, precisión y callbacks
trainer = pl.Trainer(
	max_epochs=EPOCHS_SSL,                                     # número de épocas SSL
	accelerator="gpu" if torch.cuda.is_available() else "cpu", # usa GPU si existe
	devices=1,                                                 # una GPU/CPU
	precision="16-mixed" if torch.cuda.is_available() else 32, # mixed precision en GPU
	benchmark=True,                                            # optimiza cudnn para tamaños fijos
	log_every_n_steps=10,                                      # frecuencia de logging
	gradient_clip_val=1.0,                                     # evita exploding gradients
	callbacks=[lr_monitor]                                     # callback para monitorear LR
)

# Ejecuta el loop de entrenamiento auto-supervisado
trainer.fit(timeclr_model, ssl_dl)

# Extrae el encoder ya preentrenado (sin projector) y lo pone en modo eval
encoder = timeclr_model.encoder.eval()

In [ ]:
# Almacenamos el encoder entrenado
encoder_path = f"checkpoints/encoder_ssl.pt"
torch.save(encoder.state_dict(), encoder_path)
print(f"Encoder SSL guardado en: {encoder_path}")

### **6.8 LinearProbe**

Esta clase ``LinearProbe`` implementa la fase supervisada posterior al entrenamiento auto-supervisado (el linear probing). El objetivo es evaluar o ajustar el encoder aprendido (por ejemplo, de ``TimeCLR``) sin modificar sus pesos, añadiendo una cabeza lineal de clasificación entrenable encima.
- init: Se recibe un encoder preentrenado (de ``TimeCLR``). Se congelan sus parámetros (requires_grad = False) para que no se actualicen. Se añade una capa de clasificación (``ClassifierHead``) que sí se entrena. Se define la función de pérdida (``CrossEntropy`` o ``FocalLoss``) y pesos de clase opcionales.
- forward: Pasa el ECG por el encoder (sin gradientes) y luego por la cabeza clasificadora.
- training_step, validation_step y test_step: Calcula la pérdida y métricas (accuracy, F1 macro, F1 por clase). Registra los resultados en PyTorch Lightning (self.log).
- configure_optimizers: Usa ``AdamW`` solo para optimizar la cabeza clasificadora.

En resumen: ``LinearProbe`` entrena una pequeña red clasificadora sobre un encoder congelado, permitiendo medir qué tan buena es la representación aprendida auto-supervisadamente para una tarea específica (por ejemplo, clasificación de arritmias).

In [ ]:
# ============================================================
# LinearProbe: fase supervisada tras el preentrenamiento SSL
# Entrena solo una "cabeza" de clasificación sobre un encoder congelado.
# ============================================================
class LinearProbe(pl.LightningModule):
	def __init__(
		self,
		encoder,                                  # encoder preentrenado (p. ej. TimeCLR)
		n_classes: int,                           # número de clases de salida
		lr: float = LR_LP,                        # learning rate
		wd: float = WD,                           # weight decay (regularización L2)
		class_weights: torch.Tensor | None = None # pesos por clase (opcional)
	):
		super().__init__()
		self.encoder = encoder
		self.lr, self.wd = lr, wd
		self.n_classes = n_classes

		# ------------------------------------------------------------
		# Congelamos los parámetros del encoder: no se actualizan
		# Solo se entrena la cabeza clasificadora
		# ------------------------------------------------------------
		for p in self.encoder.parameters():
			p.requires_grad = False
		
		# Definimos la cabeza de clasificación (MLP con dropout)
		self.head = ClassifierHead(
			in_dim=EMBEDDING_DIM, # tamaño del embedding del encoder
			hid=256,              # tamaño oculto intermedio
			n_classes=n_classes,  # clases de salida
			p_drop=0.4            # probabilidad de dropout
		)
		
		# ------------------------------------------------------------
		# Guardamos pesos de clase (si existen) como buffer
		# Los buffers se mueven automáticamente a GPU/CPU con el modelo
		# ------------------------------------------------------------
		if class_weights is not None:
			self.register_buffer("class_weights", class_weights.clone().float())
		else:
			self.class_weights = None
		
		# ------------------------------------------------------------
		# Selección de función de pérdida (definida globalmente)
		# ------------------------------------------------------------
		if LOSS_LP == "FocalLoss":
			self.criterion = self._focal_loss_lp
		elif LOSS_LP == "CrossEntropy":
			self.criterion = self._ce_loss_lp
		else:
			raise ValueError(f"Función de pérdida desconocida: {LOSS_LP}")

	# ============================================================
	# Utilidad: devuelve los pesos de clase en el dispositivo correcto
	# ============================================================
	def _get_weights(self, logits):
		if self.class_weights is None:
			return None
		return self.class_weights.to(device=logits.device, dtype=logits.dtype)

	# ============================================================
	# Definición de Focal Loss (versión local)
	# ============================================================
	def _focal_loss_lp(self, logits, y):
		w = self._get_weights(logits)
		return FocalLoss(
			gamma=GAMMA_LP,
			weight=w
		)(logits, y)

	# ============================================================
	# Definición de CrossEntropy Loss con label smoothing
	# ============================================================
	def _ce_loss_lp(self, logits, y):
		w = self._get_weights(logits)
		return F.cross_entropy(
			logits,
			y,
			#weight=w,           # opcionalmente aplicar pesos
			label_smoothing=0.05 # suaviza etiquetas para mejor generalización
		)

	# ============================================================
	# Forward: pasa por el encoder congelado y luego por la cabeza
	# ============================================================
	def forward(self, x):
		with torch.no_grad():   # evita gradientes en el encoder
			h = self.encoder(x) # genera embeddings fijos
		logits = self.head(h)   # los pasa por la cabeza de clasificación
		return logits

	# ============================================================
	# Función auxiliar: calcula métricas por batch
	# ============================================================
	def _compute_batch_metrics(self, logits, y, split: str):
		"""
		split: 'lp_train' o 'lp_val' para prefijar bien los nombres de logs.
		"""
		pred = logits.argmax(dim=1) # clase predicha

		# Accuracy promedio del batch
		acc = (pred == y).float().mean()
		
		# F1 por clase
		f1_per_class = []
		for c in range(self.n_classes):
			tp = ((pred == c) & (y == c)).sum().float()
			fp = ((pred == c) & (y != c)).sum().float()
			fn = ((pred != c) & (y == c)).sum().float()

			precision = tp / (tp + fp + 1e-8)
			recall = tp / (tp + fn + 1e-8)
			f1 = 2 * precision * recall / (precision + recall + 1e-8)
			f1_per_class.append(f1)

			# Guarda F1 por clase en los logs de Lightning
			self.log(
				f"{split}_f1_class_{c}",
				f1,
				on_step=False,
				on_epoch=True,
				prog_bar=False,
				logger=True,
			)

		# F1 macro: promedio de todas las clases
		f1_macro = torch.stack(f1_per_class).mean()
		return acc, f1_macro

	# ============================================================
	# Paso de entrenamiento supervisado
	# ============================================================
	def training_step(self, batch, _):
		x, y = batch
		logits = self(x)
		loss = self.criterion(logits, y)

		acc, f1_macro = self._compute_batch_metrics(logits, y, split="lp_train")

		# Registra métricas en Lightning (se muestran en barra/logs)
		self.log("lp_train_loss", loss, prog_bar=True, on_step=True, on_epoch=True, logger=True)
		self.log("lp_train_acc", acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
		self.log("lp_train_f1_macro", f1_macro, prog_bar=True, on_step=False, on_epoch=True, logger=True)

		return loss
	
	# ============================================================
	# Paso de validación
	# ============================================================
	def validation_step(self, batch, _):
		x, y = batch
		logits = self(x)
		loss = self.criterion(logits, y)

		acc, f1_macro = self._compute_batch_metrics(logits, y, split="lp_val")

		self.log("lp_val_loss", loss, prog_bar=True, on_step=True, on_epoch=True, logger=True)
		self.log("lp_val_acc", acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
		self.log("lp_val_f1_macro", f1_macro, prog_bar=True, on_step=False, on_epoch=True, logger=True)

		return loss

	# ============================================================
	# Paso de testeo (solo calcula accuracy)
	# ============================================================
	def test_step(self, batch, _):
		x, y = batch
		logits = self(x)
		pred = logits.argmax(1)
		acc = (pred == y).float().mean()
		self.log("lp_test_acc", acc, prog_bar=True)

	# ============================================================
	# Configura optimizador (solo entrena la cabeza)
	# ============================================================
	def configure_optimizers(self):
		return AdamW(self.head.parameters(), lr=self.lr, weight_decay=self.wd)

In [ ]:
# Cargamos el encoder previamente almacenado
encoder = ResNet1DBackbone(in_ch=1, emb_dim=EMBEDDING_DIM)
encoder_path = "checkpoints/encoder_ssl.pt"
encoder.load_state_dict(torch.load(encoder_path, map_location="cpu"))
encoder.eval()

In [ ]:
# ============================================================
# 1) División del dataset etiquetado en entrenamiento y validación
# ============================================================
X_tr, X_val, y_tr, y_val = train_test_split(
X_lab_train,                 # señales etiquetadas disponibles
	y_lab_train.astype(int), # etiquetas como enteros
	test_size=0.25,          # % para validación
	stratify=y_lab_train,    # mantiene proporción de clases (estratificado)
	random_state=SEED        # semilla para reproducibilidad
)

# ============================================================
# Creación de datasets etiquetados para cada split
# ============================================================
train_ds = LabeledECGDataset(
	X_tr,
	y_tr,
	transform=ecgtime_augment, # aplica augmentaciones al entrenamiento
	p_aug=0.7                  # probabilidad de aplicar augmentación
)
val_ds = LabeledECGDataset(X_val, y_val) # sin augmentación
test_ds = LabeledECGDataset(X_test, y_test.astype(int)) # dataset final de testeo

# ============================================================
# 2) Cálculo de pesos por clase para balancear el sampler
# ============================================================
y_tr_np = y_tr.astype(int)
class_counts_tr = np.bincount(y_tr_np, minlength=n_classes) # cuenta muestras por clase
print("Cantidad por clase en y_tr (0..4):", class_counts_tr)

# Parámetro alpha controla el grado de balanceo
alpha = 0.0
# Inverso de la frecuencia por clase (controlado por alpha)
inv_sampler = 1.0 / (class_counts ** alpha)
# Normaliza los pesos para que su media sea 1
class_weights_sampler = inv_sampler / inv_sampler.mean()
print("Pesos por clase para SAMPLER (0..4):", class_weights_sampler)

# Asigna a cada muestra su peso según su clase
sample_weights_np = class_weights_sampler[y_tr_np]
sample_weights = torch.from_numpy(sample_weights_np).float()
print("sample_weights shape:", sample_weights.shape)

# ============================================================
# 3) Creación de un WeightedRandomSampler para balancear las clases
# ============================================================
sampler = WeightedRandomSampler(
	weights=sample_weights,          # pesos calculados para cada muestra
	num_samples=len(sample_weights), # tamaño total del set de entrenamiento
	replacement=True                 # permite repetir muestras (sampling con reemplazo)
)

# ============================================================
# 4) DataLoaders: manejan los batches para entrenamiento y evaluación
# ============================================================

# --- Entrenamiento ---
train_dl = DataLoader(
	train_ds,
	batch_size=BATCH_LP,    # tamaño de batch para el LinearProbe
	sampler=sampler,        # usa el sampler balanceado en vez de shuffle
	shuffle=False,          # desactivado porque sampler ya decide el orden
	num_workers=NUM_WORKERS # carga de datos en paralelo
)

# --- Validación ---
val_dl = DataLoader(
	val_ds,
	batch_size=BATCH_LP,
	shuffle=False,
	num_workers=NUM_WORKERS
)

# --- Testeo ---
test_dl = DataLoader(
	test_ds,
	batch_size=BATCH_LP,
	shuffle=False,
	num_workers=NUM_WORKERS
)

In [ ]:
# ============================================================
# Callback de Early Stopping
# ============================================================
early_stop = EarlyStopping(
	monitor="lp_val_f1_macro", # métrica a monitorear (F1 macro en validación)
	mode="max",                # queremos maximizar esta métrica
	patience=8,                # número de épocas sin mejora antes de detener el entrenamiento
	min_delta=1e-4             # cambio mínimo considerado como mejora
)

# ============================================================
# Callback de Checkpointing
# ============================================================
checkpoint = ModelCheckpoint(
	monitor="lp_val_f1_macro",                       # métrica a monitorear
	mode="max",                                      # guarda el modelo con mayor F1 macro
	save_top_k=1,                                    # solo guarda el mejor modelo
	filename="lp-best-{epoch}-{lp_val_f1_macro:.4f}" # nombre del archivo guardado
)

In [ ]:
# ============================================================
# 1) Instancia del modelo Linear Probe
# ============================================================
lp = LinearProbe(
	encoder=encoder,                 # encoder preentrenado (de TimeCLR, congelado)
	n_classes=n_classes,             # número de clases a predecir
	lr=LR_LP,                        # tasa de aprendizaje del clasificador
	wd=WD,                           # regularización L2 (weight decay)
	class_weights=class_weights_loss # pesos de clase para manejar desbalance
)

# ============================================================
# 2) Configuración del Trainer de PyTorch Lightning
# ============================================================
trainer = pl.Trainer(
	max_epochs=EPOCHS_LP,                                      # número máximo de épocas supervisadas
	accelerator="gpu" if torch.cuda.is_available() else "cpu", # usa GPU si existe
	devices=1,                                                 # número de dispositivos (1 GPU o CPU)
	callbacks=[early_stop, checkpoint]                         # callbacks para early stopping y guardado del mejor modelo
)

# ============================================================
# 3) Entrenamiento con datos etiquetados (fase supervisada)
# ============================================================
trainer.fit(lp, train_dl, val_dl)

# ============================================================
# 4) Carga del mejor modelo según la métrica de validación
# ============================================================
best_lp = LinearProbe.load_from_checkpoint(
	checkpoint.best_model_path,      # ruta al mejor checkpoint guardado
	encoder=encoder,                 # se vuelve a pasar el encoder
	n_classes=num_classes,           # número de clases
	lr=LR_LP,
	wd=WD,
	class_weights=class_weights_loss
)

# ============================================================
# 5) Evaluación final (testeo)
# ============================================================
trainer.test(best_lp, test_dl)

In [ ]:
# Almacenamos nuestro linear probe entrenado
linearprobe_path = f"checkpoints/linearprobe_head_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.pt"
torch.save(lp.head.state_dict(), linearprobe_path)
print(f"Cabeza del LinearProbe guardada en: {linearprobe_path}")

### **6.9 FineTuning TimeCLR**

La clase FineTune implementa la fase de ajuste fino (fine-tuning) sobre el encoder preentrenado de TimeCLR. El objetivo es ajustar parcialmente el encoder (ya preentrenado de forma auto-supervisada) junto con una nueva cabeza de clasificación para mejorar el rendimiento en una tarea supervisada.

Resumen de funcionamiento:
- Recibe el encoder preentrenado y lo congela casi por completo, salvo su capa final (fc) si existe.
- Agrega una cabeza clasificadora (ClassifierHead) con dropout y ReLU.
- Define una función de pérdida (CrossEntropy o FocalLoss) y soporta pesos por clase.
- Durante el entrenamiento (training_step, validation_step, test_step), calcula la pérdida y métricas (accuracy, F1 macro y por clase).
- Usa un optimizador AdamW con dos grupos de parámetros:
	- uno para el encoder (con lr_enc, tasa baja)
	- otro para la cabeza (lr_head, tasa más alta).
- Aplica un scheduler CosineAnnealingLR para decaer gradualmente el learning rate.

En resumen: esta clase entrena el modelo completo (encoder + cabeza), pero con aprendizaje diferencial, refinando solo ciertas partes del encoder — una etapa típica posterior al Linear Probe dentro del flujo semi-supervisado.

In [ ]:
# ============================================================
# FineTune: fase de ajuste fino del modelo preentrenado
# ============================================================
# A diferencia del LinearProbe, aquí parte del encoder puede actualizarse,
# para adaptar mejor las representaciones auto-supervisadas a la tarea final.
# ============================================================
class FineTune(pl.LightningModule):
	def __init__(
		self,
		encoder,                                  # encoder preentrenado (de TimeCLR)
		n_classes: int,                           # número de clases de salida
		lr_enc=LR_FT_ENC,                         # learning rate para el encoder
		lr_head=LR_FT_HEAD,                       # learning rate para la cabeza
		wd=WD,                                    # weight decay (regularización L2)
		class_weights: torch.Tensor | None = None # pesos opcionales por clase
	):
		super().__init__()
		self.encoder = encoder
		self.lr_enc, self.lr_head, self.wd = lr_enc, lr_head, wd
		self.n_classes = n_classes

		# ------------------------------------------------------------
		# Congela todo el encoder, excepto su capa final "fc" (si existe)
		# Esto permite refinar parcialmente las representaciones aprendidas
		# ------------------------------------------------------------
		for p in self.encoder.parameters():
			p.requires_grad = False
		if hasattr(self.encoder, "fc"):
			for p in self.encoder.fc.parameters():
				p.requires_grad = True

		# Crea la cabeza de clasificación (MLP con normalización y dropout)
		self.head = ClassifierHead(
			in_dim=EMBEDDING_DIM,
			hid=256,
			n_classes=n_classes,
			p_drop=0.4
		)

		# Guarda los pesos de clase como buffer (se mueven con el modelo a GPU/CPU)
		if class_weights is not None:
			self.register_buffer("class_weights", class_weights.clone().float())
		else:
			self.class_weights = None
		
		# Selecciona la función de pérdida (Focal o CrossEntropy)
		if LOSS_FN == "FocalLoss":
			self.criterion = self._focal_loss_ft
		elif LOSS_FN == "CrossEntropy":
			self.criterion = self._ce_loss_ft
		else:
			raise ValueError(f"Función de pérdida desconocida: {LOSS_FN}")

	# ============================================================
	# Obtiene los pesos de clase en el dispositivo adecuado
	# ============================================================
	def _get_weights(self, logits):
		if self.class_weights is None:
			return None
		return self.class_weights.to(device=logits.device, dtype=logits.dtype)

	# ============================================================
	# Focal Loss: da más peso a ejemplos difíciles
	# ============================================================
	def _focal_loss_ft(self, logits, y):
		w = self._get_weights(logits)
		return FocalLoss(
			gamma=GAMMA_FN,
			#weight=w
		)(logits, y)

	# ============================================================
	# Cross Entropy Loss con label smoothing (suaviza etiquetas)
	# ============================================================
	def _ce_loss_ft(self, logits, y):
		w = self._get_weights(logits)
		return F.cross_entropy(
			logits,
			y,
			#weight=w,           # opcionalmente aplicar pesos
			label_smoothing=0.05 # suaviza etiquetas para mejor generalización
		)

	# ============================================================
	# Forward: pasa por encoder + cabeza clasificadora
	# ============================================================
	def forward(self, x):
		h = self.encoder(x)   # extrae embeddings
		logits = self.head(h) # produce logits de clase
		return logits

	# ============================================================
	# Cálculo de métricas por batch (accuracy, F1 macro, F1 por clase)
	# ============================================================
	def _compute_batch_metrics(self, logits, y, split: str):
		"""
		split: 'ft_train', 'ft_val' o 'ft_test' para prefijar bien los nombres de logs.
		"""
		pred = logits.argmax(dim=1) # clase predicha

		acc = (pred == y).float().mean() # accuracy promedio

		f1_per_class = []
		for c in range(self.n_classes):
			tp = ((pred == c) & (y == c)).sum().float()
			fp = ((pred == c) & (y != c)).sum().float()
			fn = ((pred != c) & (y == c)).sum().float()

			precision = tp / (tp + fp + 1e-8)
			recall = tp / (tp + fn + 1e-8)
			f1 = 2 * precision * recall / (precision + recall + 1e-8)
			f1_per_class.append(f1)

			# Guarda F1 por clase en los logs
			self.log(
				f"{split}_f1_class_{c}",
				f1,
				on_step=False,
				on_epoch=True,
				prog_bar=False,
				logger=True,
			)

		f1_macro = torch.stack(f1_per_class).mean() # promedio (macro)
		return acc, f1_macro
	
	# ============================================================
	# Paso de entrenamiento
	# ============================================================
	def training_step(self, batch, _):
		x, y = batch
		logits = self(x)
		loss = self.criterion(logits, y)

		acc, f1_macro = self._compute_batch_metrics(logits, y, split="ft_train")

		self.log("ft_train_loss", loss, prog_bar=True, on_step=True, on_epoch=True, logger=True)
		self.log("ft_train_acc", acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
		self.log("ft_train_f1_macro", f1_macro, prog_bar=True, on_step=False, on_epoch=True, logger=True)

		return loss

	# ============================================================
	# Paso de validación
	# ============================================================
	def validation_step(self, batch, _):
		x, y = batch
		logits = self(x)
		loss = self.criterion(logits, y)

		acc, f1_macro = self._compute_batch_metrics(logits, y, split="ft_val")

		self.log("ft_val_loss", loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
		self.log("ft_val_acc", acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
		self.log("ft_val_f1_macro", f1_macro, prog_bar=True, on_step=False, on_epoch=True, logger=True)
		
		return loss

	# ============================================================
	# Paso de testeo
	# ============================================================
	def test_step(self, batch, _):
		x, y = batch
		logits = self(x)
		loss = self.criterion(logits, y)

		acc, f1_macro = self._compute_batch_metrics(logits, y, split="ft_test")

		self.log("ft_test_loss", loss, prog_bar=False, on_step=False, on_epoch=True)
		self.log("ft_test_acc", acc, prog_bar=True, on_step=False, on_epoch=True)
		self.log("ft_test_f1_macro", f1_macro, prog_bar=True, on_step=False, on_epoch=True)

		return loss

	# ============================================================
	# Configura optimizador y scheduler
	# ============================================================
	def configure_optimizers(self):
		enc_params = [p for p in self.encoder.parameters() if p.requires_grad]
		head_params = [p for p in self.head.parameters() if p.requires_grad]

		param_groups = []
		if len(enc_params) > 0:
			param_groups.append({
				"params": enc_params,
				"lr": self.lr_enc,
				"weight_decay": self.wd,
			})
		if len(head_params) > 0:
			param_groups.append({
				"params": head_params,
				"lr": self.lr_head,
				"weight_decay": self.wd,
			})
		
		# Optimizador AdamW con los dos grupos de parámetros
		opt = AdamW(param_groups)

		# Scheduler: reduce LR suavemente con CosineAnnealing
		sch = CosineAnnealingLR(opt, T_max=EPOCHS_FT)
		
		return {"optimizer": opt, "lr_scheduler": sch}

In [ ]:
# ============================================================
# 1) División del dataset etiquetado en entrenamiento y validación
# ============================================================
X_tr, X_val, y_tr, y_val = train_test_split(
	X_lab_train,          # señales ECG etiquetadas disponibles
	y_lab_train,          # etiquetas correspondientes
	test_size=0.25,       # % de los datos de reserva para validación
	stratify=y_lab_train, # mantiene la proporción de clases (balance estratificado)
	random_state=SEED     # semilla para reproducibilidad
)

# ============================================================
# 2) WeightedRandomSampler para entrenamiento balanceado
# ============================================================
sampler = WeightedRandomSampler(
	weights=sample_weights,          # pesos asociados a cada muestra
	num_samples=len(sample_weights), # cantidad total de ejemplos a muestrear
	replacement=True                 # con reemplazo (permite repetir ejemplos raros)
)

# ============================================================
# 3) DataLoader de entrenamiento (Fine-Tuning)
# ============================================================
dl_tr = DataLoader(
	LabeledECGDataset(X_tr, y_tr, transform=ecgtime_augment, p_aug=0.7),
	batch_size=BATCH_FT,    # tamaño del lote
	sampler=sampler,        # sampler balanceado (en vez de shuffle)
	shuffle=False,          # no se usa shuffle cuando hay sampler
	num_workers=NUM_WORKERS # hilos paralelos para carga de datos
)

# ============================================================
# 4) DataLoader de validación (Fine-Tuning)
# ============================================================
dl_val = DataLoader(
	LabeledECGDataset(X_val, y_val),
	batch_size=BATCH_FT,
	shuffle=False,
	num_workers=NUM_WORKERS
)

# ============================================================
# 5) DataLoader de testeo (Fine-Tuning)
# ============================================================
dl_te = DataLoader(
	test_ds,
	batch_size=BATCH_FT,
	shuffle=False,
	num_workers=NUM_WORKERS
)

In [ ]:
# ============================================================
# Callback de Early Stopping
# ============================================================
early_stop = pl.callbacks.EarlyStopping(
	monitor="ft_val_f1_macro", # métrica que se va a monitorear (F1 macro en validación)
	mode="max",                # se busca maximizar esta métrica
	patience=6                 # número de épocas sin mejora antes de detener el entrenamiento
)

# ============================================================
# Callback de Checkpoint (guardado automático del mejor modelo)
# ============================================================
ckpt = pl.callbacks.ModelCheckpoint(
	monitor="ft_val_f1_macro", # métrica usada para decidir cuál modelo guardar
	mode="max",                # guarda el modelo con el mayor F1 macro
	save_top_k=1               # mantiene solo el mejor checkpoint
)

In [ ]:
# ============================================================
# 1) Instanciación del modelo FineTune
# ============================================================
ft = FineTune(
	encoder=encoder,                 # encoder preentrenado de TimeCLR (ResNet1D)
	n_classes=n_classes,             # número de clases de salida
	lr_enc=LR_FT_ENC,                # learning rate para el encoder
	lr_head=LR_FT_HEAD,              # learning rate para la cabeza
	wd=WD,                           # regularización L2 (weight decay)
	class_weights=class_weights_loss # pesos por clase (para manejar desbalance)
)

# ============================================================
# 2) Configuración del Trainer de PyTorch Lightning
# ============================================================
trainer = pl.Trainer(
	max_epochs=EPOCHS_FT,                                      # número máximo de épocas de Fine-Tuning
	accelerator="gpu" if torch.cuda.is_available() else "cpu", # usa GPU si está disponible
	devices=1,                                                 # número de dispositivos (1 GPU o CPU)
	callbacks=[early_stop, ckpt]                               # callbacks: EarlyStopping + ModelCheckpoint
)

# ============================================================
# 3) Inicialización de la cabeza del FineTune con pesos del LinearProbe
# ============================================================
linearprobe_path = "checkpoints/linearprobe_head.pt"
ft.head.load_state_dict(torch.load(linearprobe_path, map_location="cpu"))
print("FineTune.head inicializada desde LinearProbe")

# ============================================================
# 4) Entrenamiento supervisado (Fine-Tuning)
# ============================================================
trainer.fit(ft, dl_tr, dl_val)

# ============================================================
# 5) Evaluación final en el conjunto de testeo
# ============================================================
trainer.test(ft, dl_te)	

In [ ]:
# Almacenamos nuestro modelo con fine tuning
finetune_path = f"checkpoints/finetune_best.ckpt"
trainer.save_checkpoint(finetune_path)
print(f"Modelo FineTune guardado en: {finetune_path}")

### **6.10 Evaluación del Modelo**

In [ ]:
@torch.no_grad()
def evaluate_all_metrics(
	model: torch.nn.Module,
	dataloader,
	num_classes: int,
	class_names: Optional[List[str]] = None,
	device: Optional[torch.device] = None,
	normalize_cm: bool = True
) -> Dict[str, object]:
	model.eval()
	if device is None:
		device = next(model.parameters()).device

	all_preds = []
	all_probs = []
	all_true  = []

	for batch in dataloader:
		if isinstance(batch, (list, tuple)) and len(batch) == 2:
			x, y = batch
		else:
			raise ValueError("Dataloader debe entregar (x, y).")

		x = x.to(device)
		y = y.to(device)

		logits = model(x)
		probs  = F.softmax(logits, dim=1)
		preds  = probs.argmax(1)

		all_true.append(y.cpu().numpy())
		all_preds.append(preds.cpu().numpy())
		all_probs.append(probs.cpu().numpy())

	y_true = np.concatenate(all_true, axis=0)
	y_pred = np.concatenate(all_preds, axis=0)
	y_prob = np.concatenate(all_probs, axis=0)

	acc     = accuracy_score(y_true, y_pred)
	f1_mac  = f1_score(y_true, y_pred, average="macro", zero_division=0)
	rec_per_class = recall_score(y_true, y_pred, average=None, labels=np.arange(num_classes), zero_division=0)

	target_names = class_names if (class_names is not None and len(class_names) == num_classes) else None
	cls_report = classification_report(
		y_true, y_pred,
		target_names=target_names,
		zero_division=0,
		digits=4
	)

	cm = confusion_matrix(y_true, y_pred, labels=np.arange(num_classes))
	if normalize_cm:
		with np.errstate(all="ignore"):
			cm_norm = cm / cm.sum(axis=1, keepdims=True)
			cm_norm = np.nan_to_num(cm_norm)
	else:
		cm_norm = None

	roc_auc_macro = None
	try:
		unique_test_classes = np.unique(y_true)
		if num_classes == 2:
			if set(unique_test_classes.tolist()) == {0, 1}:
				roc_auc_macro = roc_auc_score(y_true, y_prob[:, 1])
			else:
				classes_sorted = np.sort(unique_test_classes)
				mapper = {c:i for i, c in enumerate(classes_sorted)}
				y_true_bin = np.vectorize(mapper.get)(y_true)
				pos_index = np.where(classes_sorted == classes_sorted.max())[0][0]
				roc_auc_macro = roc_auc_score(y_true_bin, y_prob[:, pos_index])
		else:
			if len(unique_test_classes) >= 2:
				roc_auc_macro = roc_auc_score(y_true, y_prob, average="macro", multi_class="ovr")
			else:
				roc_auc_macro = None
	except Exception as e:
		roc_auc_macro = None

	results = {
		"f1_macro": f1_mac,
		"accuracy": acc,
		"recall_per_class": rec_per_class,
		"confusion_matrix": cm,
		"confusion_matrix_normalized": cm_norm,
		"roc_auc_macro": roc_auc_macro,
		"classification_report": cls_report,
	}

	# Impresión amigable
	print("\n=== Evaluación ===")
	print(f"F1-macro: {f1_mac:.4f}")
	print(f"Accuracy: {acc:.4f}")
	if roc_auc_macro is not None:
		print(f"ROC-AUC macro (OVR): {roc_auc_macro:.4f}")
	else:
		print("ROC-AUC macro: no disponible (clases ausentes o caso no válido).")

	print("\nRecall por clase:")
	if target_names is not None:
		for i, r in enumerate(rec_per_class):
			print(f"  {target_names[i]}: {r:.4f}")
	else:
		for i, r in enumerate(rec_per_class):
			print(f"  clase {i}: {r:.4f}")

	print("\nReporte por clase:\n", cls_report)
	print("Matriz de confusión (cruda):\n", cm)
	if cm_norm is not None:
		print("Matriz de confusión (normalizada por fila):\n", np.round(cm_norm, 4))

	return results

In [ ]:
num_classes = int(np.unique(y_train[~np.isnan(y_train).astype(bool)]).size) if hasattr(y_train, "dtype") else int(np.unique(y_train).size)

class_names = [
	"(N) Latido normal",
	"(S) Latido supraventricular",
	"(V) Latido ventricular ectópico",
	"(F) Latido de fusión",
	"(Q) Latido desconocido"
]

results = evaluate_all_metrics(
	model=ft,
	dataloader=test_dl,
	num_classes=num_classes,
	class_names=class_names
)

### **6.11 Threshold Tuning**

In [ ]:
# Modo evaluación
ft.eval()
ft.to(DEVICE)

# Recolectar probabilidades y etiquetas verdaderas del conjunto de validación
all_logits, all_y = [], []

with torch.no_grad():
	for x, y in val_dl:
		x = x.to(DEVICE)
		logits = ft(x)
		all_logits.append(logits.cpu())
		all_y.append(y.cpu())

all_logits = torch.cat(all_logits)
all_y = torch.cat(all_y).numpy()

# Softmax → Probabilidades por clase
probs = torch.softmax(all_logits, dim=1).numpy()
n_classes = probs.shape[1]

# Buscar el mejor threshold por clase (maximizando F1)
best_thresholds = np.ones(n_classes) * 0.5  # inicial
for c in range(n_classes):
	best_f1, best_th = 0.0, 0.5
	for th in np.linspace(0.3, 0.9, 25):  # recorre valores posibles
		pred_c = (probs[:, c] >= th).astype(int)
		true_c = (all_y == c).astype(int)
		f1_c = f1_score(true_c, pred_c)
		if f1_c > best_f1:
			best_f1, best_th = f1_c, th
	best_thresholds[c] = best_th
	print(f"Clase {c}: threshold óptimo = {best_th:.3f}, F1 = {best_f1:.3f}")

print("\nUmbrales óptimos por clase:", np.round(best_thresholds, 3))

# Función para aplicar los thresholds a las predicciones
def predict_with_thresholds(probs, thresholds):
	adjusted = probs - thresholds[None, :]
	return adjusted.argmax(axis=1)

# Aplicar y evaluar con thresholds óptimos
y_pred = predict_with_thresholds(probs, best_thresholds)

print("\n=== Evaluación con THRESHOLDS ajustados ===")
print(classification_report(all_y, y_pred, digits=4))

In [ ]:
# Guardar thresholds para test/inferencia futura
best_thresholds_path = f"checkpoints/best_thresholds_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.npy"
np.save(best_thresholds_path, best_thresholds)

### **6.12 Escritura en CSV**

In [ ]:
@torch.no_grad()
def write_predictions_csv(model, test_dl, thresholds_path="best_thresholds.npy", out_path="preds.csv"):
	model.eval()
	device = next(model.parameters()).device

	# Cargar thresholds ajustados
	thresholds = np.load(thresholds_path)
	print("Umbrales cargados:", np.round(thresholds, 3))

	ids = []
	preds = []
	running_id = 0

	for batch in test_dl:
		if isinstance(batch, (list, tuple)) and len(batch) >= 1:
			x = batch[0]
		else:
			x = batch

		bsz = x.size(0)
		x = x.to(device)

		# Probabilidades por clase
		logits = model(x)
		p = F.softmax(logits, dim=1).cpu().numpy()

		# Aplicar thresholds por clase
		adjusted = p - thresholds[None, :]
		yhat = adjusted.argmax(axis=1)

		ids.extend(range(running_id, running_id + bsz))
		preds.extend(yhat.tolist())
		running_id += bsz

	df = pd.DataFrame({"ID": ids, "label": preds})
	df.to_csv(out_path, index=False)
	print(f"CSV de predicciones guardado en: {out_path}")

In [ ]:
write_predictions_csv(
	ft,
	test_dl,
	thresholds_path="checkpoints/best_thresholds.npy",
	out_path=f"ecg_submittions/timeclr_predictions_thresholded_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
)